# 2. Generate Converation

## Load configuration from `.env` file

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path=".env")

print(os.getenv("AZURE_OPENAI_ENDPOINT")) 
print(os.getenv("AZURE_OPENAI_MODEL"))
print(os.getenv("AZURE_OPENAI_REASONING_MODEL"))

## Setup the System Prompt for generating the dialogue

The SYSTEM_PROMPT below is used to set the context for the AI model to generate a conversation. It includes details about the podcast, the participants, and the structure of the conversation. It is combined with a user prompt later in the notebook that contains the source data the AI will use to generate the conversation.

In [ ]:
SYSTEM_PROMPT = """
You are a top-tier podcast producer tasked with crafting an engaging and entertaining sports podcast script centered on the Kansas City Chiefs, styled after local sports talk radio, based on the provided input text. The input may be unstructured or fragmented, sourced from PDFs or web pages. Your goal is to transform this content into a lively, sports-focused podcast segment filled with excitement, humor, and sports analysis.

Podcast Host Profiles:
The two podcast hosts are Skeeter and Doug, who have a friendly rivalry and a deep passion for the Kansas City Chiefs. Skeeter is known for his enthusiastic and optimistic takes, while Doug often plays the devil's advocate, providing a more pessimistic, critical perspective. Their banter is lively, humorous, and filled with inside jokes about the Chiefs.
- Skeeter: Upbeat loyalist who brings energy and optimism - even in tough losses.
- Doug: Skeptical realist who often challenges Skeeter's views, reacting sharply to poor performances, missed opportunities, and underachievement.

# Steps to Follow:
1. **Analyze the Input:**   
   Review the text thoroughly to identify key highlights, exciting sports moments, standout performances, player statistics, significant game events, and interesting anecdotes. Ignore irrelevant details or formatting irregularities, but creatively connect events from different time frames if needed to maintain narrative momentum.

2. Brainstorm Ideas:
   Within the `<scratchpad>`, develop creative ways to present sports content dynamically. Consider:
   - Energetic play-by-play recaps
   - Engaging analogies or sports metaphors
   - Humorous commentary and lively banter between hosts
   - Dramatic framing of key sports moments
   - Thought-provoking sports debates or rhetorical questions to spark listener interest

3. Craft the Dialogue:
   Create a vibrant, fast-paced conversation between two charismatic sports anchors (Host 1: Skeeter, Host 2: Doug), mirroring local sports talk radio energetic style. Incorporate:
   - A catchy opening to immediately captivate listeners
   - Playful and competitive banter between hosts
   - Authentic excitement, including spontaneous reactions ("Wow!", "Did you see that?")
   - Occasional friendly disagreements or debates over sports outcomes
   - Humor, sports clichés, and quick wit to enhance entertainment value - use these sparingly
   - Relevant quotes or reactions drawn directly from the input text
   - Maintain a balance between jokes and serious analysis, ensuring the discussion remains engaging and informative.
   Rules for the dialogue:
   - Skeeter always initiates the discussion and sets the tone
   - Doug provides color commentary, humorous insights, and reactions
   - Hosts naturally interrupt each other for realism and entertainment
   - Maintain a high-energy yet family-friendly tone suitable for all audiences
   - Conclude with a memorable wrap-up summarizing key sports highlights
   - The hosts should spend longer on more important topics, allowing for more in-depth discussion and analysis.

4. Highlight Key Moments:
   Throughout the podcast, repeatedly emphasize key sports events and insights from the input text, ensuring listeners grasp and remember the most significant moments without feeling lectured.

5. Ensure Entertainment Value:
   Maintain a lively, upbeat tone by including:
   - Surprising facts or amusing anecdotes
   - Playful banter and clever exchanges between hosts
   - Dramatic build-ups and enthusiastic descriptions of game-defining moments

6. Consider Pacing and Structure:
   Structure your script for optimal listener engagement:
   - Start immediately with a bold, engaging opening statement
   - Alternate smoothly between high-energy recaps and insightful analysis
   - Include brief pauses or slower segments to help listeners digest intense sports action
   - End energetically, perhaps teasing upcoming sports events or posing a fun challenge to listeners


Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""

## Create classes to be used for Structured Output against the LLM endpoints

Here we use the `pydantic` library to define structured output classes. These classes will help us ensure that the data returned by the AI model is in a consistent format. In this case, we define classes for two different turns with the LLM. The first turn sends raw data about a game, week, season, etc... and the LLM will extract a Topic and Topic Items. The other classes define different length dialogues which are used to generate a "script" from an LLM that can then be turned into SSML for TTS.

In [ ]:
from typing import Literal, List

from pydantic import BaseModel, Field
import os


class DialogueItem(BaseModel):
    """A single dialogue item."""

    speaker: Literal["Skeeter", "Doug"]
    text: str
    sentiment: Literal["exciting", "serious", "empathetic", "cheerful"]

class MediumDialogue(BaseModel):
    """The dialogue between the host and guest."""

    scratchpad: str
    name_of_guest: str
    dialogue: List[DialogueItem] = Field(
        ..., description="A list of dialogue items, typically between 29 to 39 items"
    )

class TopicItem(BaseModel):
    """A single topic item."""
    time: str = Field(..., description="When the topic occurred in the match")
    subject: str = Field(..., description="The subject of the topic (examples: 'Injury', 'Play', 'Score', 'Penalty')")
    activity: str = Field(..., description="The activity related to the topic")
    description: str = Field(..., description="A description of the topic")
class Topics(BaseModel):
    """A class to represent the topics of a sports podcast episode."""
    week: str = Field(..., description="The date of the topic")
    matchup: str = Field(..., description="Name of the matchup between two teams (example: New York Giants vs. Green Bay Packers)")
    topics: List[TopicItem] = Field(
        ..., description="A list of topic items"
    )

## Fetching raw data

This section loads data from the `examples` directory. The initial commit of this notebook is setup to consume a CSV file called `playerstats.csv` but there are commented out examples showing loading of simple .txt files, as well as one showing loading an array of Markdown files and merging them into a single string.

In [ ]:
import csv

#with open(os.path.join('..', 'examples', "game-recap.txt"), "r") as f:
#    input_text = f.read()
content = []
with open(os.path.join('..', 'examples', "playerstats.csv"), "r") as f:
    reader = csv.reader(f)
    for row in reader:
        line = ", ".join(cell.strip() for cell in row if cell.strip())
        if line:
            content.append(line)
input_text = "\n".join(content)

# other_updates = [os.path.join('..', 'examples', "call-center-status1.md"), os.path.join('..', 'examples', "call-center-status2.md"), os.path.join('..', 'examples', "call-center-status3.md")]
# for update in other_updates:
#     with open(update, "r") as f:
#         input_text += f.read()
#print(input_text)

## Creating a Summary (Topics Extraction) System Prompt

This section defines a system prompt that instructs the AI model to extract topics from the provided data. The topics will be used to guide the conversation in the podcast.

In [ ]:
# Let's make an initial LLM call to condition the input for the dialogue generation
SUMMARY_SYSTEM_PROMPT = """
You are a world-class sports podcast producer tasked with extracting from the provided input key pieces of data that will be used later to generate a script for a podcast episode. The input may be unstructured or messy, sourced from PDFs or web pages. Your goal is to extract the key information, things like teams, players, final scores, significant plays, injuries, and anything else an audiance of fans might be interested in. You will then group them in a chronological order so they can be used later to generate a script for a podcast episode.


# Steps to Follow:
1. **Analyze the Input:** 
    Carefullly examine the text to pull out key information like teams, players, final scores, significant plays, injuries, and anything else fans might be interested in.
2. **Group the Information:**
    Group the information chronologically by quarter, half, or period.
3. **Output the Information:**
    Output the information in a JSON format, without any additional text or explanation.

Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""

## Generate Topics and Podcast dialogue

The section below makes two calls to LLMs. The first call is to extract topics from the raw data. The second call generates a dialogue based on the topics extracted in the first call. The dialogue is structured to include multiple turns, with each turn representing a part of the conversation between the podcast participants.

LLM calls are abstracted (implementation can be found in the `utils.py` file). The `call_reasoning_llm` function is used to call an Azure OpenAI Reasioning Model, while the `call_llm` function is used to call a standard Azure OpenAI Model. 

In the initial commit an experiment was being run to use the `call_reasoning_llm` function to extract topics (given the source data had times and order matters to the dialog), while the `call_llm` function was used to generate the dialogue. 

In [ ]:
from utils import call_llm, call_reasoning_llm

topic_extraction = call_reasoning_llm(SUMMARY_SYSTEM_PROMPT, input_text, Topics)
topic_extraction.to_dict()
topic_dialog_feeder = topic_extraction.model_dump_json()

modified_system_prompt = SYSTEM_PROMPT
modified_system_prompt += "\n\nAim for a moderate length, about 3-5 minutes."
modified_system_prompt += "\n\nOUTPUT LANGUAGE <IMPORTANT>: The the podcast should be English."

# Call the LLM for the first time
first_draft_dialogue = call_llm(modified_system_prompt, topic_dialog_feeder, MediumDialogue)

## Output the first draft dialogue

The section outputs the first draft of the dialogue generated by the AI model. This dialogue is structured according to the defined classes and includes multiple turns between the podcast participants.

In [ ]:
first_draft_dialogue.to_dict()

## Refine the dialogue

This section demonstrates how to refine the dialogue generated by the AI model. It uses the `call_llm` function to make a second call to the AI model, providing it with the initial draft and asking it to improve or refine the conversation. The refined dialogue is then outputted for review.

In [ ]:
# Call the LLM a second time to improve the dialogue
system_prompt_with_dialogue = f"{modified_system_prompt}\n\nHere is the first draft of the dialogue you provided:\n\n{first_draft_dialogue.model_dump_json()}."
final_dialogue = call_llm(system_prompt_with_dialogue, "Please improve the dialogue. Make it more natural and engaging. Remeber this isn't play-by-play but a recap of a game that has happened.", MediumDialogue)

## Output the final draft dialogue

The section outputs the final draft of the dialogue generated by the AI model. This dialogue is structured according to the defined classes and includes multiple turns between the podcast participants.

In [ ]:
final_dialogue.to_dict()

## Extract the dialogue from the final draft and output it to a JSON file

The final two sections of the notebook extract the dialogue from the final draft and output it to a JSON file. This file can be used for further processing, such as converting the dialogue to SSML for text-to-speech (TTS) conversion.

In [ ]:
import json
result = json.loads(final_dialogue.choices[0].message.content)
result["dialogue"]

In [ ]:

with open("game-recap-script.json", "w") as f:
    f.write(json.dumps(result["dialogue"], indent=4, ensure_ascii=False))